# Compartments

In [1]:
r_enz = 'HindIII'
rep = 'T0'

In [2]:
from pytadbit.parsers.hic_parser import load_hic_data_from_reads

/home/fransua/.miniconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [ ]:
hic_data = load_hic_data_from_reads(
    'results/fragment/{0}_{1}/03_filtering/valid_reads12_{0}_{1}.tsv'.format(r_enz, rep),
    100000)